## ipcoal: simulation and analysis of genealogies and gene trees

The `ipcoal` Python package provides a convenient framework for simulating and analyzing genealogies and inferred gene trees under complex demographic scenarios. You can generate demographic models representing population histories, species trees, or networks, using a newick file that can be visualized in `toytree`. The model parameters will be parsed by `ipcoal` to define a simulation framework in `msprime` to generate a distribution of genealogies from which SNPs, loci, or chromosomes can be simulated. The simulated sequence data can be saved to disk in a variety of formats, or, gene tree analyses can be automated and parallelized to infer empirical gene trees on the simulated data. The resulting true genealogies, summary statistics, and inferred trees are returned by `ipcoal` is a Pandas DataFrame for further statistical analysis. 

### Required software
All required software can be installed with the following conda command. 

In [1]:
# conda install ipcoal -c conda-forge

In [2]:
import toytree
import ipcoal

### The main functions of *ipcoal*
You start by initializing a `Model` class object by providing a species tree/network and additional optional model parameters (e.g., Ne, migration, mutation rate, recombination rate). Then you can simulate either loci or SNPs on the genealogies produced under this model. `ipcoal` makes it easy to either write the sequence data to files under a variety of formats, or to perform phylogenetic inference on the sequence data directly. You can then compare true simulated genealogies to the inferred trees. 

In [4]:
# init a model Class object for simulations
model = ipcoal.Model(tree=toytree.rtree.unittree(5, treeheight=1e6))

# simulate N unlinked SNPs (will run until N snps are produced)
model.sim_snps(10)

# simulate N loci of len L 
model.sim_loci(10, 10)

# access the genealogies in a table
model.df

# access the sequence data in an array
model.seqs

# infer a tree for every locus
model.infer_gene_trees(method='raxml')

# infer a species tree from the concatenated data
model.infer_concatenation_tree(method='raxml')

# compare inferred gene trees to true genealogies (called after infer_gene_trees)
model.calculate_rf_dists()

# write loci as separate phylip files to a directory
model.write_loci_to_phylip(outdir="tests/")

# write concatenated loci or snps to a phylip file
model.write_seqs_to_phylip("test.phy")

TypeError: __init__() got an unexpected keyword argument 'method'

In [9]:
import numpy as np
model.df["inferred_tree"] = np.nan

In [10]:
model.df

,loc,start,end,nbps,nsnps,genealogy,inferred_tree
0,0,0,1,1,1,"((4:675854.06941967690364,5:675854.06941967690...",NaN
1,1,0,1,1,1,"((4:669066.33026261103805,5:669066.33026261103...",NaN
2,2,0,1,1,1,"((3:676410.99631966697052,(1:344280.7795671388...",NaN
3,3,0,1,1,1,"((3:672374.30080978525802,(1:357792.8470557146...",NaN
4,4,0,1,1,1,"((3:670907.10137404419947,(1:378181.3519377416...",NaN
5,5,0,1,1,1,"((4:673649.28592275176197,5:673649.28592275176...",NaN
6,6,0,1,1,1,"((4:666742.77196698694024,5:666742.77196698694...",NaN
7,7,0,1,1,1,"((3:667281.22066993557382,(1:355027.4748672380...",NaN
8,8,0,1,1,1,"((3:676287.27356363122817,(1:405425.6421063020...",NaN
9,9,0,1,1,1,"((3:671895.83903710462619,(1:335132.6239233383...",NaN


### Define a species/population tree
Node heights should be in units of generations. 

In [12]:
# generate a random 6-tip tree with root height of 1M generations
tree = toytree.rtree.coaltree(6, seed=222).mod.node_scale_root_height(1e6)

# draw tree showing idx labels
tree.draw(tree_style='c');

<svg class="toyplot-canvas-Canvas" height="250.0px" id="tde4a3deed2e142f3ae13ebe53ed7e7b5" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 350.0 250.0" width="350.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> idx: 0
name: r0
dist: 87185.6719
support: 100
height: -0.0000 0 idx: 1
name: r1
dist: 87185.6719
support: 100
height: -0.0000 1 idx: 2
name: r2
dist: 171542.2928
support: 100
height: -0.0000 2 idx: 3
name: r3
dist: 171542.2928
support: 100
height: -0.0000 3 idx: 4
name: r4
dist: 389555.4181
support: 100
height: 0.0000 4 idx: 5
name: r5
dist: 389555.4181
support: 100
height: 0.0000 5 idx: 6
name: 6
dist: 504042.8998
support: 100
height: 87185.6719 6 idx: 7
name: 7
dist: 419686.2788
support: 100
height: 171542.2928 7 idx: 8
name: 8
dist: 408771.4283
support: 100
height: 591228.5717 8 idx: 9
name: 9
dist: 610444.5819
support: 100
height: 389555.4181 9 idx: 10
name: 10
dist: 195.7821
support: 100
height: 1000000.0000 10 0 500000 1000000

### Define an ipcoal simulation model. 
Here you can define the demographic model by setting a global Ne value (overrides Ne values stored to the tree), and setting the mutation and recombination rates. You can define a admixture scenarios using a simple syntax provided by a list of tuples. In each tuple you list the (source, dest, edge_prop, rate), where edge_prop is a float value of the proportion of the length of the shared edge between two taxa from recent to the past at which the migration pulse took place. In other words, if you set this to (7, 4, 0.5, 0.1) then 10% of the population of 7 will migrate into population 4 (backwards in time) at the midpoint of the shared edge between them. 

In [20]:
sim = ipcoal.Model(
    tree,
    Ne=1e6,
    mut=1e-8,
    recomb=1e-9,
    seed=123,
    admixture_edges=[(6, 4, 0.5, 0.1)],
    debug=True,
)

migration: edge(6->4) time(238370.545, 238370.545), rate(0.100, 0.100)
div time:       1000000  4  0 (9) (8)
div time:        389555  5  4 (5) (4)
div time:        591228  2  0 (7) (6)
div time:        171542  3  2 (3) (2)
div time:         87185  1  0 (1) (0)
mig pulse:       238370  6  4 0.100
pop: Ne:1000000, mut:1.00E-08


### Simulate genealogies and sequences for N independent loci of length L
Because our simulation includes recombination each locus may represent multiple genealogical histories. You can see this in the dataframe below where loc 0 is represented by 5 genealogies. 

In [21]:
# run the simulation
sim.run(nloci=10, nsites=500)

In [22]:
# view the genealogies and their summary stats
sim.df.head(10)

,loc,start,end,nbps,nsnps,genealogy
0,0,0,104,104,7,"(r4:2676110.71983299450949,(r3:1119930.5197808..."
1,0,104,234,130,11,"((r0:2208173.71173396287486,r4:2208173.7117339..."
2,0,234,257,23,3,"((r4:2208173.71173396287486,(r0:1349517.661202..."
3,0,257,438,181,20,"((r3:2208173.71173396287486,(r0:1865495.190384..."
4,0,438,500,62,6,"((r3:2208173.71173396287486,r0:2208173.7117339..."
5,1,0,146,146,13,"((r2:1310300.31713711423799,r3:1310300.3171371..."
6,1,146,163,17,3,"((r1:840626.75758881168440,(r4:802206.53257372..."
7,1,163,175,12,2,"((r2:1310300.31713711423799,r3:1310300.3171371..."
8,1,175,212,37,2,"(r0:2790036.30116437561810,((r1:840626.7575888..."
9,1,212,242,30,3,"((r1:840626.75758881168440,(r4:802206.53257372..."


In [23]:
# view the sequence array for the first locus (first 20 bp)
sim.seqs[0, :, :20]

array([[1, 1, 1, 1, 0, 2, 2, 1, 1, 0, 1, 3, 0, 0, 2, 3, 0, 0, 3, 3],
       [1, 1, 1, 1, 0, 2, 2, 1, 1, 0, 1, 3, 0, 0, 2, 3, 0, 0, 3, 3],
       [1, 1, 1, 1, 0, 2, 2, 1, 1, 0, 1, 3, 0, 0, 2, 3, 0, 0, 3, 3],
       [1, 1, 1, 1, 0, 2, 2, 1, 1, 0, 1, 3, 0, 0, 2, 3, 0, 0, 3, 3],
       [1, 1, 1, 1, 1, 2, 2, 1, 1, 0, 1, 3, 0, 0, 2, 3, 0, 0, 3, 3],
       [1, 1, 1, 1, 0, 2, 2, 1, 1, 0, 1, 3, 0, 0, 2, 3, 0, 0, 3, 3]],
      dtype=uint8)

In [24]:
# write all loci as separate phylip files to a directory
sim.write_loci_to_phylip()

In [25]:
# write all loci concatenated to a single sequence file
sim.write_seqs_to_phylip()

### Visualize genealogical variation using toytree

In [26]:
# load a multitree object from first 5 genealogies
mtre = toytree.mtree(sim.df.genealogy.tolist())

# draw trees from the first locus
#  with 'shared_axis' to show diff in heights
#  with 'fixed_order' to show diff in topology (relative to first tree)
mtre.draw_tree_grid(
    start=0,
    ncols=4, 
    nrows=1,
    shared_axis=True,
    fixed_order=True,
    edge_type='c',
);

# draw trees from the second locus
mtre.draw_tree_grid(
    start=5,
    ncols=4, 
    nrows=1,
    shared_axis=True,
    fixed_order=True,
    edge_type='c',
);

<svg class="toyplot-canvas-Canvas" height="250.0px" id="t40a22d61ff3b44f796f9724eeeb6730d" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 700.0 250.0" width="700.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> r5 r1 r2 r0 r3 r4 r5 r1 r2 r0 r3 r4 r5 r1 r2 r0 r3 r4 r5 r1 r2 r0 r3 r4 0 1338055 2676111

<svg class="toyplot-canvas-Canvas" height="250.0px" id="t1a156eedba4144aa8d9b69d0f4b5b469" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 700.0 250.0" width="700.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> r5 r1 r2 r0 r3 r4 r5 r1 r2 r0 r3 r4 r5 r1 r2 r0 r3 r4 r5 r1 r2 r0 r3 r4 0 1623035 3246070

### Simulate N unlinked SNPs

In [29]:
# simulate N unlinked SNPs
sim._run_snps(100)

In [30]:
# the genealogies for each SNP are stored in .df
sim.df.head()

,loc,start,end,nbps,nsnps,genealogy
0,0,0,1,1,1,"(2:3897021.81761665828526,(1:1461933.132032580..."
1,1,0,1,1,1,"(((3:1066284.61568715423346,5:1066284.61568715..."
2,2,0,1,1,1,"((5:393672.12702140613692,6:393672.12702140613..."
3,3,0,1,1,1,"(1:3774465.04241469409317,((5:1143798.46381672..."
4,4,0,1,1,1,"((5:424907.42707304155920,6:424907.42707304155..."


In [31]:
# the snp array is stores in .seqs
sim.seqs[:, :20]

array([[3, 3, 0, 3, 1, 0, 2, 2, 1, 1, 3, 3, 3, 0, 1, 2, 3, 2, 3, 0],
       [2, 1, 1, 1, 1, 0, 2, 2, 2, 1, 1, 3, 0, 0, 1, 2, 2, 2, 0, 0],
       [3, 3, 0, 1, 0, 3, 1, 1, 1, 1, 3, 3, 3, 1, 1, 1, 2, 2, 0, 3],
       [3, 1, 0, 1, 1, 0, 2, 2, 1, 1, 1, 3, 3, 0, 0, 1, 2, 2, 0, 3],
       [3, 3, 0, 1, 1, 0, 2, 2, 1, 1, 1, 3, 3, 1, 1, 1, 2, 2, 0, 3],
       [3, 3, 0, 1, 1, 0, 2, 2, 1, 2, 1, 1, 3, 1, 1, 1, 2, 3, 0, 3]],
      dtype=uint8)

In [ ]:
# write the snps array as a phylip file
sim.write_seqs_to_phylip()

In [ ]:
# 

### Infer a tree at each locus with `iqtree`.

In [24]:
mod.infer_trees(method = 'iqtree')

In [25]:
mod.df

,locus_idx,starts,stops,bps,newicks,cumulative_bps,inferred_trees
0,0,0.000000,399.114605,399,"(r4:171878.93510039325338,(r3:135044.601240783...",399,"(r0:0.0026263194,r1:0.0036614296,((r2:0.009439..."
1,0,399.114605,2003.711868,1605,"(r4:171878.93510039325338,(r3:135044.601240783...",2004,"(r0:0.0026263194,r1:0.0036614296,((r2:0.009439..."
2,0,2003.711868,3215.876200,1212,"(r4:171878.93510039325338,((r0:31028.726571306...",3216,"(r0:0.0026263194,r1:0.0036614296,((r2:0.009439..."
3,0,3215.876200,4029.008186,813,"(r4:171878.93510039325338,(r2:135044.601240783...",4029,"(r0:0.0026263194,r1:0.0036614296,((r2:0.009439..."
4,0,4029.008186,4424.136472,395,"((r3:73409.16564434758038,(r0:31028.7265713068...",4424,"(r0:0.0026263194,r1:0.0036614296,((r2:0.009439..."
5,0,4424.136472,4469.888225,46,"(r4:203004.80203346943017,(r2:151000.469034697...",4470,"(r0:0.0026263194,r1:0.0036614296,((r2:0.009439..."
6,0,4469.888225,5000.000000,530,"(r4:203004.80203346943017,(r2:151000.469034697...",5000,"(r0:0.0026263194,r1:0.0036614296,((r2:0.009439..."
0,1,0.000000,68.697338,69,"(r4:170791.99710473464802,(r3:142028.118303298...",5069,"(r0:0.0028067520,r1:0.0034128597,(r2:0.0070096..."
1,1,68.697338,341.245211,272,"(r4:170791.99710473464802,(r3:125979.354911937...",5341,"(r0:0.0028067520,r1:0.0034128597,(r2:0.0070096..."
2,1,341.245211,382.042768,41,"(r4:170791.99710473464802,(r3:125979.354911937...",5382,"(r0:0.0028067520,r1:0.0034128597,(r2:0.0070096..."


### We can also look at the raw sequences:

In [26]:
mod.seqs.shape

(10, 5, 5000)

In [27]:
mod.seqs[0]

array([[2, 1, 0, ..., 0, 1, 2],
       [2, 1, 0, ..., 0, 1, 2],
       [2, 1, 0, ..., 0, 1, 2],
       [2, 1, 0, ..., 0, 1, 2],
       [2, 1, 0, ..., 0, 1, 2]], dtype=int8)

### If we want to generate SNPs in a simcat-like format, we can do that separately:

In [28]:
snps = mod._run_snps(1000)

In [29]:
snps

array([116,  16,  14, ...,  40,  39,  16])

In [30]:
for idx, mat in enumerate(snps.reshape((5,16,16))):
    toyplot.matrix(mat, label="Matrix " + str(idx), colorshow=True);

<svg class="toyplot-canvas-Canvas" height="600px" id="tebf8eb6aa48d4c0b8eabd0a148c9e599" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 600 600" width="600px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> Matrix 0 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 0 116.000000 16.000000 14.000000 14.000000 15.000000 6.000000 0.000000 0.000000 13.000000 0.000000 6.000000 0.000000 9.000000 0.000000 0.000000 12.000000 1 2.000000 0.000000 0.000000 0.000000 0.000000 3.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 2 4.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 7.000000 0.000000 0.000000 0.000000 0.000000 0.000000 3 3.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 1.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 6.000000 4 1.000000 1.000000 0.000000 0.000000 0.000000 5.000000 1.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 5 6.000000 8.000000 0.000000 0.000000 20.000000 130.000000 20.000000 17.000000 0.000000 11.000000 13.000000 0.000000 0.000000 5.000000 0.000000 8.000000 6 0.000000 0.000000 0.000000 0.000000 0.000000 2.000000 0.000000 0.000000 0.000000 0.000000 3.000000 0.000000 0.000000 0.000000 0.000000 0.000000 7 0.000000 0.000000 0.000000 0.000000 0.000000 10.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 2.000000 8 3.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 6.000000 0.000000 0.000000 0.000000 0.000000 0.000000 9 0.000000 0.000000 0.000000 0.000000 0.000000 5.000000 0.000000 0.000000 0.000000 0.000000 3.000000 0.000000 0.000000 0.000000 0.000000 0.000000 10 4.000000 0.000000 6.000000 0.000000 0.000000 8.000000 11.000000 0.000000 15.000000 13.000000 128.000000 14.000000 0.000000 0.000000 9.000000 7.000000 11 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 2.000000 0.000000 0.000000 0.000000 0.000000 8.000000 12 6.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 3.000000 13 0.000000 0.000000 0.000000 0.000000 0.000000 2.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 2.000000 14 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 5.000000 0.000000 0.000000 0.000000 0.000000 5.000000 15 4.000000 0.000000 0.000000 11.000000 0.000000 8.000000 0.000000 11.000000 0.000000 0.000000 8.000000 10.000000 15.000000 14.000000 18.000000 126.000000 0 50 100 150

<svg class="toyplot-canvas-Canvas" height="600px" id="t7a3264b6699e4f40bc2a9340ecd3350e" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 600 600" width="600px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> Matrix 1 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 0 33.000000 34.000000 44.000000 49.000000 15.000000 6.000000 0.000000 0.000000 15.000000 0.000000 3.000000 1.000000 10.000000 0.000000 0.000000 11.000000 1 2.000000 0.000000 0.000000 0.000000 0.000000 3.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 2 4.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 7.000000 0.000000 0.000000 0.000000 0.000000 0.000000 3 3.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 1.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 6.000000 4 2.000000 0.000000 0.000000 0.000000 0.000000 6.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 5 6.000000 8.000000 0.000000 0.000000 38.000000 39.000000 61.000000 49.000000 0.000000 11.000000 13.000000 0.000000 0.000000 6.000000 0.000000 7.000000 6 0.000000 0.000000 0.000000 0.000000 0.000000 2.000000 0.000000 0.000000 0.000000 0.000000 3.000000 0.000000 0.000000 0.000000 0.000000 0.000000 7 0.000000 0.000000 0.000000 0.000000 0.000000 10.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 2.000000 8 3.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 6.000000 0.000000 0.000000 0.000000 0.000000 0.000000 9 0.000000 0.000000 0.000000 0.000000 0.000000 5.000000 0.000000 0.000000 0.000000 0.000000 3.000000 0.000000 0.000000 0.000000 0.000000 0.000000 10 5.000000 0.000000 5.000000 0.000000 0.000000 8.000000 11.000000 0.000000 56.000000 36.000000 35.000000 43.000000 1.000000 1.000000 9.000000 5.000000 11 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 2.000000 0.000000 0.000000 0.000000 0.000000 8.000000 12 6.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 3.000000 13 0.000000 0.000000 0.000000 0.000000 0.000000 2.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 2.000000 14 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 5.000000 0.000000 0.000000 0.000000 0.000000 5.000000 15 4.000000 0.000000 0.000000 11.000000 0.000000 8.000000 0.000000 11.000000 1.000000 1.000000 7.000000 9.000000 51.000000 44.000000 41.000000 37.000000 0 20 40 60

<svg class="toyplot-canvas-Canvas" height="600px" id="t739780217b0f43b78c11f25f7087192f" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 600 600" width="600px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> Matrix 2 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 0 36.000000 31.000000 41.000000 45.000000 13.000000 9.000000 0.000000 0.000000 13.000000 0.000000 6.000000 1.000000 11.000000 0.000000 0.000000 15.000000 1 2.000000 0.000000 0.000000 0.000000 0.000000 3.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 2 4.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 7.000000 0.000000 0.000000 0.000000 0.000000 0.000000 3 3.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 7.000000 4 1.000000 0.000000 0.000000 0.000000 1.000000 5.000000 0.000000 0.000000 0.000000 1.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 5 9.000000 15.000000 2.000000 0.000000 35.000000 24.000000 51.000000 44.000000 0.000000 12.000000 21.000000 0.000000 0.000000 13.000000 0.000000 12.000000 6 0.000000 0.000000 0.000000 0.000000 0.000000 2.000000 0.000000 0.000000 0.000000 0.000000 3.000000 0.000000 0.000000 0.000000 0.000000 0.000000 7 0.000000 0.000000 0.000000 0.000000 0.000000 10.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 2.000000 8 3.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 6.000000 0.000000 0.000000 0.000000 0.000000 0.000000 9 0.000000 0.000000 0.000000 0.000000 0.000000 5.000000 0.000000 0.000000 0.000000 0.000000 3.000000 0.000000 0.000000 0.000000 0.000000 0.000000 10 5.000000 0.000000 13.000000 1.000000 0.000000 10.000000 11.000000 0.000000 56.000000 34.000000 24.000000 40.000000 1.000000 1.000000 12.000000 7.000000 11 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 2.000000 0.000000 0.000000 0.000000 0.000000 8.000000 12 6.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 3.000000 13 0.000000 0.000000 0.000000 0.000000 0.000000 2.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 2.000000 14 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 5.000000 0.000000 0.000000 0.000000 0.000000 5.000000 15 8.000000 2.000000 0.000000 9.000000 0.000000 10.000000 0.000000 12.000000 0.000000 1.000000 9.000000 16.000000 48.000000 40.000000 39.000000 31.000000 0 20 40 60

<svg class="toyplot-canvas-Canvas" height="600px" id="tb68da0d0b6504c609741173cd85db7e4" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 600 600" width="600px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> Matrix 3 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 0 9.000000 30.000000 41.000000 45.000000 12.000000 4.000000 0.000000 0.000000 11.000000 0.000000 3.000000 0.000000 10.000000 0.000000 0.000000 4.000000 1 14.000000 1.000000 0.000000 0.000000 1.000000 8.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 1.000000 2 13.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 2.000000 0.000000 10.000000 1.000000 0.000000 0.000000 0.000000 0.000000 3 9.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 1.000000 0.000000 0.000000 17.000000 4 7.000000 0.000000 0.000000 0.000000 1.000000 8.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 5 3.000000 15.000000 2.000000 0.000000 35.000000 17.000000 51.000000 44.000000 0.000000 13.000000 8.000000 0.000000 0.000000 12.000000 0.000000 5.000000 6 0.000000 0.000000 0.000000 0.000000 0.000000 11.000000 0.000000 0.000000 0.000000 0.000000 16.000000 0.000000 0.000000 0.000000 0.000000 0.000000 7 0.000000 0.000000 0.000000 0.000000 0.000000 5.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 1.000000 0.000000 9.000000 8 7.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 1.000000 0.000000 5.000000 0.000000 0.000000 0.000000 0.000000 0.000000 9 0.000000 0.000000 0.000000 0.000000 0.000000 13.000000 0.000000 0.000000 0.000000 0.000000 11.000000 0.000000 0.000000 0.000000 0.000000 0.000000 10 1.000000 0.000000 13.000000 1.000000 0.000000 2.000000 11.000000 0.000000 55.000000 33.000000 11.000000 40.000000 0.000000 1.000000 11.000000 2.000000 11 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 1.000000 8.000000 0.000000 1.000000 0.000000 1.000000 13.000000 12 10.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 11.000000 13 0.000000 0.000000 0.000000 0.000000 0.000000 10.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 11.000000 14 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 1.000000 12.000000 0.000000 1.000000 0.000000 0.000000 9.000000 15 4.000000 2.000000 0.000000 9.000000 0.000000 2.000000 0.000000 12.000000 0.000000 0.000000 2.000000 16.000000 47.000000 40.000000 39.000000 10.000000 0 20 40 60

<svg class="toyplot-canvas-Canvas" height="600px" id="t54e489d572764431b2ebb6583b1f3fd2" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 600 600" width="600px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> Matrix 4 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 0 10.000000 30.000000 41.000000 45.000000 13.000000 4.000000 0.000000 0.000000 11.000000 0.000000 3.000000 0.000000 10.000000 0.000000 0.000000 4.000000 1 14.000000 1.000000 0.000000 0.000000 1.000000 10.000000 0.000000 0.000000 0.000000 1.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 2 13.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 2.000000 0.000000 9.000000 1.000000 0.000000 0.000000 0.000000 0.000000 3 9.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 1.000000 0.000000 0.000000 14.000000 4 8.000000 0.000000 0.000000 0.000000 0.000000 8.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 5 3.000000 15.000000 2.000000 0.000000 35.000000 10.000000 51.000000 44.000000 0.000000 12.000000 8.000000 0.000000 0.000000 12.000000 0.000000 5.000000 6 0.000000 0.000000 0.000000 0.000000 0.000000 11.000000 0.000000 0.000000 0.000000 0.000000 16.000000 0.000000 0.000000 0.000000 0.000000 0.000000 7 0.000000 0.000000 0.000000 0.000000 0.000000 5.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 1.000000 0.000000 9.000000 8 8.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 1.000000 0.000000 5.000000 0.000000 0.000000 0.000000 0.000000 0.000000 9 0.000000 0.000000 0.000000 0.000000 0.000000 10.000000 0.000000 0.000000 0.000000 0.000000 11.000000 0.000000 0.000000 0.000000 0.000000 0.000000 10 1.000000 0.000000 13.000000 1.000000 0.000000 2.000000 11.000000 0.000000 55.000000 33.000000 15.000000 40.000000 0.000000 1.000000 11.000000 2.000000 11 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 1.000000 8.000000 0.000000 1.000000 0.000000 1.000000 10.000000 12 7.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 11.000000 13 0.000000 0.000000 0.000000 0.000000 0.000000 18.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 12.000000 14 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 1.000000 9.000000 0.000000 1.000000 0.000000 0.000000 9.000000 15 4.000000 2.000000 0.000000 9.000000 0.000000 2.000000 0.000000 12.000000 0.000000 0.000000 2.000000 16.000000 47.000000 40.000000 39.000000 16.000000 0 20 40 60

### If we want our model to have an admixture edge, we can do that too.

In [31]:
adedge = (0, 4, 0.5, 0.4)
mod = phymsim.Model(tree,
                    Ne = 10000, 
                    mut=1e-7,
                   recomb=9e-9,
                   admixture_edges=adedge)

In [32]:
mod.run(num_loci = 10,
        size = 5000)

In [33]:
snps_admixed = mod._run_snps(1000)

In [34]:
for idx, mat in enumerate(snps_admixed.reshape((5,16,16))):
    toyplot.matrix(mat, label="Matrix " + str(idx), colorshow=True);

<svg class="toyplot-canvas-Canvas" height="600px" id="tf928804a16a343518b1a371d7765e4fd" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 600 600" width="600px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> Matrix 0 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 0 84.000000 22.000000 17.000000 16.000000 12.000000 8.000000 0.000000 0.000000 11.000000 0.000000 4.000000 0.000000 8.000000 0.000000 0.000000 4.000000 1 8.000000 0.000000 0.000000 0.000000 1.000000 22.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 2 7.000000 0.000000 1.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 18.000000 0.000000 0.000000 0.000000 0.000000 1.000000 3 6.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 1.000000 0.000000 0.000000 29.000000 4 11.000000 1.000000 0.000000 0.000000 0.000000 8.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 5 5.000000 13.000000 0.000000 0.000000 19.000000 75.000000 11.000000 17.000000 0.000000 14.000000 3.000000 0.000000 0.000000 10.000000 0.000000 3.000000 6 0.000000 0.000000 0.000000 0.000000 0.000000 10.000000 0.000000 0.000000 1.000000 0.000000 14.000000 0.000000 0.000000 0.000000 0.000000 0.000000 7 0.000000 0.000000 0.000000 0.000000 0.000000 4.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 3.000000 0.000000 20.000000 8 25.000000 1.000000 5.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 4.000000 0.000000 0.000000 0.000000 0.000000 0.000000 9 0.000000 0.000000 0.000000 0.000000 0.000000 17.000000 0.000000 0.000000 0.000000 0.000000 3.000000 0.000000 0.000000 0.000000 0.000000 0.000000 10 3.000000 0.000000 10.000000 0.000000 0.000000 3.000000 11.000000 0.000000 17.000000 16.000000 65.000000 18.000000 0.000000 0.000000 13.000000 8.000000 11 0.000000 0.000000 0.000000 0.000000 0.000000 1.000000 0.000000 0.000000 0.000000 0.000000 5.000000 1.000000 0.000000 0.000000 0.000000 14.000000 12 18.000000 0.000000 1.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 5.000000 13 0.000000 0.000000 0.000000 0.000000 0.000000 19.000000 0.000000 0.000000 0.000000 1.000000 0.000000 0.000000 0.000000 0.000000 0.000000 4.000000 14 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 18.000000 3.000000 0.000000 0.000000 0.000000 11.000000 15 2.000000 0.000000 0.000000 6.000000 1.000000 5.000000 0.000000 11.000000 0.000000 0.000000 3.000000 11.000000 17.000000 11.000000 14.000000 77.000000 0 30 60 90

<svg class="toyplot-canvas-Canvas" height="600px" id="ta6aa049d04ed41908f00812c0df0e4d9" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 600 600" width="600px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> Matrix 1 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 0 43.000000 35.000000 28.000000 33.000000 11.000000 9.000000 0.000000 0.000000 12.000000 0.000000 3.000000 0.000000 7.000000 0.000000 0.000000 5.000000 1 8.000000 0.000000 0.000000 0.000000 20.000000 3.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 2 8.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 15.000000 0.000000 3.000000 0.000000 1.000000 0.000000 0.000000 0.000000 3 6.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 20.000000 0.000000 1.000000 9.000000 4 1.000000 11.000000 0.000000 0.000000 1.000000 7.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 5 6.000000 12.000000 0.000000 0.000000 27.000000 44.000000 21.000000 30.000000 0.000000 15.000000 2.000000 0.000000 0.000000 11.000000 0.000000 2.000000 6 0.000000 0.000000 0.000000 0.000000 0.000000 10.000000 0.000000 0.000000 0.000000 10.000000 4.000000 1.000000 0.000000 0.000000 0.000000 0.000000 7 0.000000 0.000000 0.000000 0.000000 0.000000 4.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 18.000000 0.000000 5.000000 8 8.000000 0.000000 23.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 4.000000 0.000000 0.000000 0.000000 0.000000 0.000000 9 0.000000 0.000000 0.000000 0.000000 0.000000 1.000000 16.000000 0.000000 0.000000 0.000000 3.000000 0.000000 0.000000 0.000000 0.000000 0.000000 10 1.000000 0.000000 12.000000 0.000000 0.000000 2.000000 12.000000 0.000000 28.000000 23.000000 41.000000 24.000000 0.000000 0.000000 14.000000 7.000000 11 0.000000 0.000000 0.000000 0.000000 0.000000 1.000000 0.000000 0.000000 0.000000 0.000000 6.000000 0.000000 0.000000 0.000000 10.000000 4.000000 12 5.000000 0.000000 0.000000 14.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 5.000000 13 0.000000 0.000000 0.000000 0.000000 1.000000 6.000000 0.000000 12.000000 0.000000 0.000000 0.000000 1.000000 0.000000 0.000000 0.000000 4.000000 14 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 2.000000 19.000000 0.000000 0.000000 0.000000 11.000000 15 2.000000 0.000000 0.000000 6.000000 1.000000 4.000000 0.000000 12.000000 0.000000 0.000000 4.000000 10.000000 26.000000 28.000000 27.000000 38.000000 0 10 20 30 40

<svg class="toyplot-canvas-Canvas" height="600px" id="t7e021cc7f97d4f02bfe5c553e810775f" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 600 600" width="600px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> Matrix 2 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 0 29.000000 31.000000 26.000000 29.000000 18.000000 12.000000 0.000000 0.000000 15.000000 0.000000 5.000000 1.000000 11.000000 1.000000 0.000000 8.000000 1 9.000000 0.000000 0.000000 0.000000 19.000000 3.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 2 7.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 16.000000 0.000000 3.000000 0.000000 1.000000 0.000000 0.000000 0.000000 3 7.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 19.000000 0.000000 1.000000 9.000000 4 1.000000 10.000000 0.000000 0.000000 1.000000 8.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 5 7.000000 17.000000 0.000000 0.000000 26.000000 36.000000 21.000000 29.000000 0.000000 12.000000 2.000000 0.000000 0.000000 17.000000 0.000000 3.000000 6 0.000000 1.000000 0.000000 0.000000 0.000000 10.000000 0.000000 0.000000 0.000000 9.000000 4.000000 1.000000 0.000000 0.000000 0.000000 0.000000 7 0.000000 0.000000 0.000000 0.000000 0.000000 7.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 15.000000 0.000000 5.000000 8 8.000000 0.000000 17.000000 0.000000 0.000000 0.000000 1.000000 0.000000 0.000000 0.000000 9.000000 0.000000 0.000000 0.000000 0.000000 0.000000 9 0.000000 0.000000 0.000000 0.000000 0.000000 1.000000 16.000000 0.000000 0.000000 0.000000 3.000000 0.000000 0.000000 0.000000 0.000000 0.000000 10 7.000000 0.000000 13.000000 0.000000 0.000000 3.000000 16.000000 0.000000 22.000000 22.000000 34.000000 21.000000 0.000000 0.000000 16.000000 10.000000 11 0.000000 0.000000 0.000000 0.000000 0.000000 1.000000 0.000000 0.000000 0.000000 0.000000 5.000000 0.000000 0.000000 0.000000 11.000000 4.000000 12 5.000000 0.000000 0.000000 13.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 1.000000 0.000000 0.000000 0.000000 5.000000 13 0.000000 0.000000 0.000000 0.000000 1.000000 6.000000 0.000000 13.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 4.000000 14 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 2.000000 16.000000 0.000000 0.000000 0.000000 14.000000 15 6.000000 0.000000 0.000000 14.000000 0.000000 5.000000 0.000000 11.000000 0.000000 0.000000 3.000000 14.000000 23.000000 27.000000 28.000000 27.000000 0 10 20 30 40

<svg class="toyplot-canvas-Canvas" height="600px" id="t37b2406d6b22430c96ef9dc84e18317e" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 600 600" width="600px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> Matrix 3 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 0 21.000000 30.000000 26.000000 28.000000 18.000000 4.000000 0.000000 0.000000 15.000000 0.000000 2.000000 1.000000 11.000000 1.000000 0.000000 4.000000 1 12.000000 1.000000 0.000000 0.000000 19.000000 11.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 2 11.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 16.000000 0.000000 6.000000 0.000000 0.000000 0.000000 0.000000 0.000000 3 8.000000 0.000000 0.000000 1.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 20.000000 0.000000 1.000000 13.000000 4 6.000000 10.000000 0.000000 0.000000 1.000000 13.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 5 2.000000 17.000000 0.000000 0.000000 26.000000 21.000000 21.000000 29.000000 0.000000 11.000000 0.000000 0.000000 0.000000 16.000000 0.000000 1.000000 6 0.000000 1.000000 0.000000 0.000000 0.000000 14.000000 0.000000 0.000000 0.000000 10.000000 6.000000 1.000000 0.000000 0.000000 0.000000 0.000000 7 0.000000 0.000000 0.000000 0.000000 0.000000 13.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 16.000000 0.000000 7.000000 8 9.000000 0.000000 19.000000 0.000000 0.000000 0.000000 1.000000 0.000000 0.000000 0.000000 15.000000 0.000000 0.000000 0.000000 0.000000 0.000000 9 0.000000 0.000000 0.000000 0.000000 0.000000 4.000000 17.000000 0.000000 0.000000 0.000000 11.000000 0.000000 0.000000 0.000000 0.000000 0.000000 10 6.000000 0.000000 11.000000 0.000000 0.000000 1.000000 15.000000 0.000000 22.000000 22.000000 12.000000 21.000000 0.000000 0.000000 16.000000 3.000000 11 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 13.000000 0.000000 0.000000 0.000000 11.000000 11.000000 12 7.000000 0.000000 0.000000 13.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 1.000000 0.000000 0.000000 0.000000 6.000000 13 1.000000 0.000000 0.000000 0.000000 1.000000 10.000000 0.000000 13.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 11.000000 14 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 1.000000 0.000000 0.000000 5.000000 16.000000 0.000000 0.000000 1.000000 13.000000 15 3.000000 0.000000 0.000000 14.000000 0.000000 1.000000 0.000000 10.000000 0.000000 0.000000 0.000000 14.000000 23.000000 27.000000 27.000000 20.000000 0 10 20 30

<svg class="toyplot-canvas-Canvas" height="600px" id="t2c5711040e0049b1b1f845583de8f9a1" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 600 600" width="600px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> Matrix 4 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 0 14.000000 40.000000 43.000000 41.000000 18.000000 5.000000 1.000000 0.000000 14.000000 0.000000 7.000000 2.000000 11.000000 1.000000 0.000000 4.000000 1 11.000000 1.000000 0.000000 0.000000 1.000000 15.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 2 11.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 1.000000 0.000000 7.000000 0.000000 0.000000 0.000000 0.000000 0.000000 3 7.000000 0.000000 0.000000 1.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 9.000000 4 13.000000 0.000000 0.000000 0.000000 1.000000 12.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 5 3.000000 17.000000 0.000000 0.000000 45.000000 10.000000 37.000000 41.000000 0.000000 11.000000 0.000000 0.000000 0.000000 16.000000 0.000000 1.000000 6 0.000000 0.000000 0.000000 0.000000 0.000000 14.000000 0.000000 1.000000 0.000000 1.000000 5.000000 0.000000 0.000000 0.000000 0.000000 0.000000 7 0.000000 0.000000 0.000000 0.000000 0.000000 10.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 1.000000 0.000000 6.000000 8 8.000000 0.000000 2.000000 0.000000 0.000000 0.000000 0.000000 0.000000 1.000000 0.000000 10.000000 0.000000 0.000000 0.000000 0.000000 0.000000 9 0.000000 0.000000 0.000000 0.000000 0.000000 12.000000 1.000000 0.000000 0.000000 0.000000 11.000000 0.000000 0.000000 0.000000 0.000000 0.000000 10 6.000000 1.000000 11.000000 0.000000 0.000000 1.000000 15.000000 0.000000 37.000000 31.000000 9.000000 38.000000 0.000000 0.000000 15.000000 6.000000 11 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 13.000000 0.000000 1.000000 0.000000 1.000000 18.000000 12 8.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 6.000000 13 1.000000 0.000000 0.000000 0.000000 0.000000 9.000000 0.000000 1.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 11.000000 14 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 8.000000 0.000000 0.000000 0.000000 2.000000 10.000000 15 4.000000 0.000000 0.000000 14.000000 0.000000 4.000000 0.000000 10.000000 0.000000 0.000000 0.000000 14.000000 42.000000 42.000000 38.000000 18.000000 0 10 20 30 40

#### notes from 10/3 meeting below.

In [ ]:
# siulate data a DF and seqarray (both stored to mod); loci can be invariant...
mod.run(...)

# alternate method for fast simulation of SNPs, makes DF and seqarray; loops to ensure nsnps=nloci
mod.run_snps(...)

# fastest simcat alternative (specialized for ensure SNPs and don't mess with DFs)
mod._run_snps(...)

In [ ]:
# look at DF 
mod.df

# look at seq arr 0
mod.seqs[0]



In [ ]:
# OPTIONAL
# write CSV to file (optinoally)
mod.df.to_csv("filname.csv")

# write phylips to dir (calls write_phy() internally?)
mod.write_seqs(path="...", concat=True)
mod.write_snps(path="...", seed=123)


In [ ]:
# infer trees for each locus 
mod.infer_trees(method="raxml")


In [ ]:
# look at results
mod.df



In [11]:
mod.seqs

{'r1': array([3, 0, 3, ..., 2, 2, 3], dtype=int8),
 'r3': array([3, 0, 3, ..., 2, 2, 3], dtype=int8),
 'r2': array([3, 0, 2, ..., 2, 2, 3], dtype=int8),
 'r0': array([3, 0, 2, ..., 2, 2, 3], dtype=int8),
 'r4': array([3, 0, 2, ..., 2, 2, 3], dtype=int8)}

In [ ]:
mod.write_phy(loc=0, path=...)

### goals
1. restructure Class object for API mode
2. complete write_seqs, write_phy, and infert_trees methods.
3. Run example code (init object, .run(), .infer_tree(), .df.to_csv()) for a couple data sets (trees, Nes, admixture) for simcat
4. write data for simcat to analyze...
5. Test simcat on getting results correct

## Now make the node values correspond to thetas:

In [75]:
idx = 0
Nes=[10000,20000,30000,40000,50000,60000,70000,80000,90000]
for node in tree.treenode.traverse():
    node.add_feature('Ne', Nes[idx])
    idx+=1

tree height: 4145.004


<svg class="toyplot-canvas-Canvas" height="275.0px" id="t937f7c47fe274bf28a11f781f9ce58b7" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 255.0 275.0" width="255.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> 0 1 2 3 4 idx: 0
name: 0
dist: 1386.2760
support: 100
height: 0.0000
Ne: 90000 0 idx: 1
name: 1
dist: 1386.2760
support: 100
height: 0.0000
Ne: 80000 1 idx: 2
name: 2
dist: 2095.9880
support: 100
height: 0.0000
Ne: 60000 2 idx: 3
name: 3
dist: 2889.2440
support: 100
height: 0.0000
Ne: 40000 3 idx: 4
name: 4
dist: 4145.0040
support: 100
height: 0.0000
Ne: 20000 4 idx: 5
name: 5
dist: 709.7120
support: 100
height: 1386.2760
Ne: 70000 5 idx: 6
name: 6
dist: 793.2560
support: 100
height: 2095.9880
Ne: 50000 6 idx: 7
name: 7
dist: 1255.7600
support: 100
height: 2889.2440
Ne: 30000 7 idx: 8
name: 8
dist: 2.0000
support: 100
height: 4145.0040
Ne: 10000 8

In [76]:
tree.draw(node_labels=tree.get_node_values("Ne", True, True), node_sizes=30, tip_labels_align=True);

<svg class="toyplot-canvas-Canvas" height="275.0px" id="t1d9e7e9c88af48cdb46e4552a736aa47" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 260.0 275.0" width="260.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> r0 r1 r2 r3 r4 90000 80000 60000 40000 20000 70000 50000 30000 10000

In [77]:
norm_thetas = list(np.array(tree.get_edge_values("Ne"))/10000)
tree.draw(edge_widths=norm_thetas,tip_labels=True,);

<svg class="toyplot-canvas-Canvas" height="275.0px" id="tb084f7eac14b4635babc38026ae6c7d3" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 260.0 275.0" width="260.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> r0 r1 r2 r3 r4

In [78]:
mod = phymsim.Model(tree,
                    Ne = None, 
                    mut=1e-8,
                   recomb=1e-9)

In [79]:
mod._get_demography()

[{'type': 'mass_migration', 'time': 1386, 'source': 1, 'dest': 0, 'proportion': 1.0},
 {'type': 'population_parameters_change', 'time': 1386, 'growth_rate': None, 'initial_size': 70000, 'population': 0},
 {'type': 'mass_migration', 'time': 2095, 'source': 2, 'dest': 0, 'proportion': 1.0},
 {'type': 'population_parameters_change', 'time': 2095, 'growth_rate': None, 'initial_size': 50000, 'population': 0},
 {'type': 'mass_migration', 'time': 2889, 'source': 3, 'dest': 0, 'proportion': 1.0},
 {'type': 'population_parameters_change', 'time': 2889, 'growth_rate': None, 'initial_size': 30000, 'population': 0},
 {'type': 'mass_migration', 'time': 4145, 'source': 4, 'dest': 0, 'proportion': 1.0},
 {'type': 'population_parameters_change', 'time': 4145, 'growth_rate': None, 'initial_size': 10000, 'population': 0}]

In [80]:
[i.initial_size for i in mod._get_popconfig()] # this tells us the size of each population index

[90000, 80000, 60000, 40000, 20000]

In [81]:
gt_df2, seqs2 = mod.run_locus(50000,seqgen=True)
gt_df2

,starts,stops,bps,newicks
0,0.000000,19455.908127,19456,"(r0:17304.66686415578806,((r1:6991.97506137920..."
1,19455.908127,24868.687562,5413,"((r1:6991.97506137920209,r4:6991.9750613792020..."
2,24868.687562,27339.247434,2470,"((r1:6991.97506137920209,r4:6991.9750613792020..."
3,27339.247434,31949.423471,4610,"((r1:6991.97506137920209,r4:6991.9750613792020..."
4,31949.423471,34001.191550,2052,"((r1:6991.97506137920209,r4:6991.9750613792020..."
5,34001.191550,43708.421009,9707,"((r1:6991.97506137920209,r4:6991.9750613792020..."
6,43708.421009,47286.784474,3579,"((r1:6991.97506137920209,r4:6991.9750613792020..."
7,47286.784474,50000.000000,2713,"((r1:6991.97506137920209,r4:6991.9750613792020..."


In [89]:
(np.array(list(range(5,20)))-3)/2

array([1. , 1.5, 2. , 2.5, 3. , 3.5, 4. , 4.5, 5. , 5.5, 6. , 6.5, 7. ,
       7.5, 8. ])

In [91]:
np.array(gt_df2)

array([[0.0, 19455.908127002396, 19456,
        '(r0:17304.66686415578806,((r1:6991.97506137920209,r4:6991.97506137920209):9186.02880066223588,(r2:7070.72458629240009,r3:7070.72458629240009):9107.27927574903879):1126.66300211435009);'],
       [19455.908127002396, 24868.687562381077, 5413,
        '((r1:6991.97506137920209,r4:6991.97506137920209):31581.69362858660679,(r0:17304.66686415578806,(r2:7070.72458629240009,r3:7070.72458629240009):10233.94227786338888):21269.00182581002082);'],
       [24868.687562381077, 27339.24743424618, 2470,
        '((r1:6991.97506137920209,r4:6991.97506137920209):31581.69362858660679,(r3:7070.72458629240009,(r0:6768.01376796623390,r2:6768.01376796623390):302.71081832616619):31502.94410367340970);'],
       [27339.24743424618, 31949.423470522608, 4610,
        '((r1:6991.97506137920209,r4:6991.97506137920209):76064.11481685213221,(r3:7070.72458629240009,(r0:6768.01376796623390,r2:6768.01376796623390):302.71081832616619):75985.36529193892784);'],
       [3

In [93]:
gt_df2['starts']

0        0.000000
1    19455.908127
2    24868.687562
3    27339.247434
4    31949.423471
5    34001.191550
6    43708.421009
7    47286.784474
Name: starts, dtype: float64

In [94]:
import h5py

In [149]:
here= h5py.File("./testit2.hdf5")

In [150]:
seqs = here.create_group('seqs')

In [153]:
for i in seqs2.keys():
    print(i)

r0
r1
r4
r2
r3


In [154]:
for i in seqs2.keys():
    seqs.create_dataset(i,data=seqs2[i])

In [99]:
writeit.create_dataset('starts',shape=(len(gt_df2['starts']),),data = gt_df2['starts'])

<HDF5 dataset "starts": shape (8,), type "<f8">

In [115]:
dt = h5py.special_dtype(vlen=str)

dset = writeit.create_dataset('newicks1', (len(gt_df2['newicks']),), dtype=dt)

In [116]:
dset = gt_df2['newicks']

In [110]:
toytree.tree(dset[0])

In [123]:
writeit['newicks3'] = gt_df2['newicks'].astype('S')

In [125]:
list(writeit['newicks3'])

[b'(r0:17304.66686415578806,((r1:6991.97506137920209,r4:6991.97506137920209):9186.02880066223588,(r2:7070.72458629240009,r3:7070.72458629240009):9107.27927574903879):1126.66300211435009);',
 b'((r1:6991.97506137920209,r4:6991.97506137920209):31581.69362858660679,(r0:17304.66686415578806,(r2:7070.72458629240009,r3:7070.72458629240009):10233.94227786338888):21269.00182581002082);',
 b'((r1:6991.97506137920209,r4:6991.97506137920209):31581.69362858660679,(r3:7070.72458629240009,(r0:6768.01376796623390,r2:6768.01376796623390):302.71081832616619):31502.94410367340970);',
 b'((r1:6991.97506137920209,r4:6991.97506137920209):76064.11481685213221,(r3:7070.72458629240009,(r0:6768.01376796623390,r2:6768.01376796623390):302.71081832616619):75985.36529193892784);',
 b'((r1:6991.97506137920209,r4:6991.97506137920209):95115.58412385180418,(r3:7070.72458629240009,(r0:6768.01376796623390,r2:6768.01376796623390):302.71081832616619):95036.83459893859981);',
 b'((r1:6991.97506137920209,r4:6991.97506137920